In [2]:
import os
import json
import random
import numpy as np
import tensorflow as tf

2021-12-15 18:10:56.986513: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [4]:
# Name of directory for grids
directory_name = './data/big/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    try:
        loaded_data = json.load(f)
    except Exception:
        continue
        
    data = random.sample(loaded_data, 30)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        if len(i['gridworld']) != 2500:
            print(file_name)
            continue
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()
    

In [5]:
print(direct_count)

[699, 1874, 4115, 5312]


In [6]:
# Name of directory for grids
directory_name = './data/bigtest/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [7]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 50, 50) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 50, 50) )
train_in_locals = np.reshape( x_train_locals, (-1, 5, 5) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 50, 50) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 50, 50) )
test_in_locals = np.reshape( x_test_locals, (-1, 5, 5) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [8]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (50,50) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [9]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (50,50) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [10]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (5,5) )
flatten_local = tf.keras.layers.Flatten()( local_input )

In [11]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_local, flatten_position])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 5025), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")


In [12]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 16, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

2021-12-15 18:11:40.878189: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-15 18:11:40.878965: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-15 18:11:40.906122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 18:11:40.906978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 3GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 9 deviceMemorySize: 2.95GiB deviceMemoryBandwidth: 178.99GiB/s
2021-12-15 18:11:40.907057: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-12-15 18:11:40.913492: I tensorflow/stream_executor/platform/defau

In [13]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, local_input, position_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [14]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [15]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

2021-12-15 18:11:41.470623: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-15 18:11:41.471192: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3199980000 Hz
2021-12-15 18:11:41.533616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-12-15 18:11:41.713937: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


0	385	0	0
0	1068	0	0
1	2344	0	0
0	3028	0	0


In [16]:
# Train the model
history = model.fit( [train_in_grid, train_in_locals, train_in_position], train_out, validation_data=([test_in_grid, test_in_locals, test_in_position], test_out), epochs = 7 )

Epoch 1/7
375/375 [==============================] - 2s 3ms/step - loss: 0.9447 - accuracy: 0.6085 - val_loss: 0.2954 - val_accuracy: 0.9188
Epoch 2/7
375/375 [==============================] - 1s 2ms/step - loss: 0.2829 - accuracy: 0.9090 - val_loss: 0.2412 - val_accuracy: 0.9215
Epoch 3/7
375/375 [==============================] - 1s 2ms/step - loss: 0.2232 - accuracy: 0.9369 - val_loss: 0.1158 - val_accuracy: 0.9665
Epoch 4/7
375/375 [==============================] - 1s 2ms/step - loss: 0.1583 - accuracy: 0.9514 - val_loss: 0.1150 - val_accuracy: 0.9635
Epoch 5/7
375/375 [==============================] - 1s 2ms/step - loss: 0.1268 - accuracy: 0.9601 - val_loss: 0.1157 - val_accuracy: 0.9593
Epoch 6/7
375/375 [==============================] - 1s 2ms/step - loss: 0.1022 - accuracy: 0.9655 - val_loss: 0.0859 - val_accuracy: 0.9717
Epoch 7/7
375/375 [==============================] - 1s 2ms/step - loss: 0.1526 - accuracy: 0.9599 - val_loss: 0.0805 - val_accuracy: 0.9741


In [17]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

354	19	2	10
40	1001	2	25
17	3	2323	2
29	24	4	2971


In [18]:
# model.save('./models/agent1_NN')

In [21]:
print(history.history.get('accuracy'))

[0.7366666793823242, 0.8790833353996277, 0.9435833096504211, 0.953416645526886, 0.953499972820282, 0.9670833349227905, 0.9590833187103271]
